# Single source shortest path
Given a graph and a source vertex, the single source shortest path (SSSP) algorithm finds a path from the source vertex to every other vertex in the graph, such that the sum of the weights of the path's edges is minimized.

SSSP was added in MADlib 1.10 and grouping was added in 1.11.

In [3]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [5]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


[(u'MADlib version: 1.11-dev, git revision: rel/v1.9.1-74-g1cc0efc, cmake configuration time: Fri Apr 14 23:56:52 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0',)]

# 1.  Create vertex and edge tables

In [6]:
%%sql 
DROP TABLE IF EXISTS vertex, edge;

CREATE TABLE vertex(
        id INTEGER
        );

CREATE TABLE edge(
        src INTEGER,
        dest INTEGER,
        weight FLOAT8
        );

INSERT INTO vertex VALUES
(0),
(1),
(2),
(3),
(4),
(5),
(6),
(7);

INSERT INTO edge VALUES
(0, 1, 1.0),
(0, 2, 1.0),
(0, 4, 10.0),
(1, 2, 2.0),
(1, 3, 10.0),
(2, 3, 1.0),
(2, 5, 1.0),
(2, 6, 3.0),
(3, 0, 1.0),
(4, 0, -2.0),
(5, 6, 1.0),
(6, 7, 1.0);

SELECT * FROM edge ORDER BY src, dest;

Done.
Done.
Done.
8 rows affected.
12 rows affected.
12 rows affected.


[(0, 1, 1.0),
 (0, 2, 1.0),
 (0, 4, 10.0),
 (1, 2, 2.0),
 (1, 3, 10.0),
 (2, 3, 1.0),
 (2, 5, 1.0),
 (2, 6, 3.0),
 (3, 0, 1.0),
 (4, 0, -2.0),
 (5, 6, 1.0),
 (6, 7, 1.0)]

# 2.  Calculate the shortest paths from vertex 0

In [7]:
%%sql
DROP TABLE IF EXISTS out, out_summary;

SELECT madlib.graph_sssp(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertex id column (NULL means use default naming)
                         'edge',        -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         0,             -- Source vertex for path calculation
                         'out');        -- Output table of shortest paths

SELECT * FROM out ORDER BY id;

Done.
1 rows affected.
8 rows affected.


[(0, 0.0, 0),
 (1, 1.0, 0),
 (2, 1.0, 0),
 (3, 2.0, 2),
 (4, 10.0, 0),
 (5, 2.0, 2),
 (6, 3.0, 5),
 (7, 4.0, 6)]

# 3.  Get the shortest path to vertex 5

In [9]:
%%sql
DROP TABLE IF EXISTS out_path;
SELECT madlib.graph_sssp_get_path('out',5, 'out_path');
SELECT * FROM out_path;

Done.
1 rows affected.
1 rows affected.


[([0, 2, 5],)]

# 4.  Custom column names
Now let's do a similar example except using different column names in the tables (i.e., not the defaults). Create the vertex and edge tables:

In [10]:
%%sql 
DROP TABLE IF EXISTS vertex_alt, edge_alt;
CREATE TABLE vertex_alt AS SELECT id AS v_id FROM vertex;
CREATE TABLE edge_alt AS SELECT src AS e_src, dest, weight AS e_weight FROM edge;
SELECT * FROM edge_alt ORDER BY e_src, dest;

Done.
8 rows affected.
12 rows affected.
12 rows affected.


[(0, 1, 1.0),
 (0, 2, 1.0),
 (0, 4, 10.0),
 (1, 2, 2.0),
 (1, 3, 10.0),
 (2, 3, 1.0),
 (2, 5, 1.0),
 (2, 6, 3.0),
 (3, 0, 1.0),
 (4, 0, -2.0),
 (5, 6, 1.0),
 (6, 7, 1.0)]

# 5.0  Get the shortest path from vertex 1

In [12]:
%%sql
DROP TABLE IF EXISTS out_alt, out_alt_summary;

SELECT madlib.graph_sssp(
                         'vertex_alt',                  -- Vertex table
                         'v_id',                        -- Vertix id column (NULL means use default naming)
                         'edge_alt',                    -- Edge table
                         'src=e_src, weight=e_weight',  -- Edge arguments (NULL means use default naming)
                         1,                             -- Source vertex for path calculation
                         'out_alt');                    -- Output table of shortest paths

SELECT * FROM out_alt ORDER BY v_id;

Done.
1 rows affected.
8 rows affected.


[(0, 4.0, 3),
 (1, 0.0, 1),
 (2, 2.0, 1),
 (3, 3.0, 2),
 (4, 14.0, 0),
 (5, 3.0, 2),
 (6, 4.0, 5),
 (7, 5.0, 6)]

# 6.0 Grouping
Create a graph with 2 groups and find SSSP for each group:

In [13]:
%%sql
DROP TABLE IF EXISTS edge_gr;

CREATE TABLE edge_gr AS
(
  SELECT *, 0 AS grp FROM edge
  UNION
  SELECT *, 1 AS grp FROM edge WHERE src < 6 AND dest < 6
);

INSERT INTO edge_gr VALUES
(4,5,-20,1);

SELECT * FROM edge_gr ORDER BY grp, src, dest;

Done.
21 rows affected.
1 rows affected.
22 rows affected.


[(0, 1, 1.0, 0),
 (0, 2, 1.0, 0),
 (0, 4, 10.0, 0),
 (1, 2, 2.0, 0),
 (1, 3, 10.0, 0),
 (2, 3, 1.0, 0),
 (2, 5, 1.0, 0),
 (2, 6, 3.0, 0),
 (3, 0, 1.0, 0),
 (4, 0, -2.0, 0),
 (5, 6, 1.0, 0),
 (6, 7, 1.0, 0),
 (0, 1, 1.0, 1),
 (0, 2, 1.0, 1),
 (0, 4, 10.0, 1),
 (1, 2, 2.0, 1),
 (1, 3, 10.0, 1),
 (2, 3, 1.0, 1),
 (2, 5, 1.0, 1),
 (3, 0, 1.0, 1),
 (4, 0, -2.0, 1),
 (4, 5, -20.0, 1)]

In [14]:
%%sql
DROP TABLE IF EXISTS out_gr, out_gr_summary;
SELECT madlib.graph_sssp(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertex id column (NULL means use default naming)
                         'edge_gr',     -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         0,             -- Source vertex for path calculation
                         'out_gr',      -- Output table of shortest paths
                         'grp'          -- Grouping columns
);

SELECT * FROM out_gr ORDER BY grp, id;

Done.
1 rows affected.
14 rows affected.


[(0, 0, 0.0, 0),
 (0, 1, 1.0, 0),
 (0, 2, 1.0, 0),
 (0, 3, 2.0, 2),
 (0, 4, 10.0, 0),
 (0, 5, 2.0, 2),
 (0, 6, 3.0, 5),
 (0, 7, 4.0, 6),
 (1, 0, 0.0, 0),
 (1, 1, 1.0, 0),
 (1, 2, 1.0, 0),
 (1, 3, 2.0, 2),
 (1, 4, 10.0, 0),
 (1, 5, -10.0, 4)]

Find the path to vertex 5 for every group:

In [15]:
%%sql
DROP TABLE IF EXISTS out_gr_path;
SELECT madlib.graph_sssp_get_path('out_gr',5,'out_gr_path');
SELECT * FROM out_gr_path ORDER BY grp;

Done.
1 rows affected.
2 rows affected.


[(0, [0, 2, 5]), (1, [0, 4, 5])]